## Getting dataset and fine tune it

In [17]:
import pandas as pd
import numpy as np

In [18]:
df = pd.read_csv("all_kindle_review .csv")

In [19]:
df.head()

,Unnamed: 0.1,Unnamed: 0,asin,helpful,rating,reviewText,reviewTime,reviewerID,reviewerName,summary,unixReviewTime
0,0,11539,B0033UV8HI,"[8, 10]",3,"Jace Rankin may be short, but he's nothing to ...","09 2, 2010",A3HHXRELK8BHQG,Ridley,Entertaining But Average,1283385600
1,1,5957,B002HJV4DE,"[1, 1]",5,Great short read. I didn't want to put it dow...,"10 8, 2013",A2RGNZ0TRF578I,Holly Butler,Terrific menage scenes!,1381190400
2,2,9146,B002ZG96I4,"[0, 0]",3,I'll start by saying this is the first of four...,"04 11, 2014",A3S0H2HV6U1I7F,Merissa,Snapdragon Alley,1397174400
3,3,7038,B002QHWOEU,"[1, 3]",3,Aggie is Angela Lansbury who carries pocketboo...,"07 5, 2014",AC4OQW3GZ919J,Cleargrace,very light murder cozy,1404518400
4,4,1776,B001A06VJ8,"[0, 1]",4,I did not expect this type of book to be in li...,"12 31, 2012",A3C9V987IQHOQD,Rjostler,Book,1356912000


In [20]:
df = df[["reviewText", "rating"]]

In [21]:
df.head()

,reviewText,rating
0,"Jace Rankin may be short, but he's nothing to ...",3
1,Great short read. I didn't want to put it dow...,5
2,I'll start by saying this is the first of four...,3
3,Aggie is Angela Lansbury who carries pocketboo...,3
4,I did not expect this type of book to be in li...,4


In [22]:
df.shape

(12000, 2)

In [23]:
df.isnull().sum()

reviewText    0
rating        0
dtype: int64

In [24]:
df["rating"].unique()

array([3, 5, 4, 2, 1])

In [25]:
df["rating"].value_counts()

rating
5    3000
4    3000
3    2000
2    2000
1    2000
Name: count, dtype: int64

## Preprocessing & Cleaning

In [26]:
df["rating"] = df["rating"].apply(lambda x: 0 if x < 3 else 1)

In [27]:
df["rating"].unique()

array([1, 0])

In [28]:
df["reviewText"] = df["reviewText"].str.lower()

In [29]:
df.head()

,reviewText,rating
0,"jace rankin may be short, but he's nothing to ...",1
1,great short read. i didn't want to put it dow...,1
2,i'll start by saying this is the first of four...,1
3,aggie is angela lansbury who carries pocketboo...,1
4,i did not expect this type of book to be in li...,1


In [30]:
import re
import nltk
from bs4 import BeautifulSoup
from nltk.corpus import stopwords
nltk.download("stopwords")

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\shiva\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [31]:
df["reviewText"] = df["reviewText"].apply(lambda x: re.sub('[^a-z A-Z 0-9]+', "", x))
df["reviewText"] = df["reviewText"].apply(lambda x: " ".join([y for y in x.split() if y not in stopwords.words("english")]))
df["reviewText"] = df["reviewText"].apply(lambda x: re.sub(r'(http|https|ftp|ssh)://([\w_-]+(?:(?:\.[\w_-]+)+))([\w.,@?^=%&:/~+#-]*[\w@?^=%&/~+#-])?', '' , str(x)))
df['reviewText'] = df['reviewText'].apply(lambda x: BeautifulSoup(x, 'lxml').get_text())
df['reviewText'] = df['reviewText'].apply(lambda x: " ".join(x.split()))

In [32]:
df.head()

,reviewText,rating
0,jace rankin may short hes nothing mess man hau...,1
1,great short read didnt want put read one sitti...,1
2,ill start saying first four books wasnt expect...,1
3,aggie angela lansbury carries pocketbooks inst...,1
4,expect type book library pleased find price right,1


In [33]:
from nltk.stem import WordNetLemmatizer

In [34]:
lemma = WordNetLemmatizer()

In [35]:
def lemma_words(text):
    return " ".join([lemma.lemmatize(word) for word in text.split()])

In [36]:
df['reviewText']=df['reviewText'].apply(lambda x:lemma_words(x))

In [37]:
df.head()

,reviewText,rating
0,jace rankin may short he nothing mess man haul...,1
1,great short read didnt want put read one sitti...,1
2,ill start saying first four book wasnt expecti...,1
3,aggie angela lansbury carry pocketbook instead...,1
4,expect type book library pleased find price right,1


## Train test Split

In [38]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df["reviewText"], df["rating"], test_size=.23)  

In [48]:
from sklearn.feature_extraction.text import CountVectorizer
vect = CountVectorizer()
X_train_bow = vect.fit_transform(X_train).toarray()
X_test_bow = vect.transform(X_test).toarray()

In [49]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf=TfidfVectorizer()
X_train_tfidf=tfidf.fit_transform(X_train).toarray()
X_test_tfidf=tfidf.transform(X_test).toarray()


In [50]:
from sklearn.naive_bayes import GaussianNB
nb_model_bow=GaussianNB().fit(X_train_bow,y_train)
nb_model_tfidf=GaussianNB().fit(X_train_tfidf,y_train)

In [51]:
from sklearn.metrics import confusion_matrix,accuracy_score,classification_report


In [53]:
y_pred_bow=nb_model_bow.predict(X_test_bow)
y_pred_tfidf=nb_model_bow.predict(X_test_tfidf)

In [54]:
confusion_matrix(y_test,y_pred_bow)

array([[ 594,  340],
       [ 816, 1010]])

In [ ]:
print("BOW accuracy: ",accuracy_score(y_test,y_pred_bow))

BOW accuracy:  0.5811594202898551


In [56]:
confusion_matrix(y_test,y_pred_tfidf)

array([[ 586,  348],
       [ 811, 1015]])

In [ ]:
print("TFIDF accuracy: "    ,accuracy_score(y_test,y_pred_tfidf))

TFIDF accuracy:  0.5800724637681159
